In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib
import pandas
import abcutils

## Load and Synthesize Data from CSV

In [ ]:
df = abcutils.load_and_synthesize_csv('sample_summaries.csv.gz')

In [ ]:
df.head()

Demonstrate how basic data filters are applied and filtered views are presented to this analysis script.

In [ ]:
filters = [
    df['darshan_app'] == 'ior', # only want IOR jobs
    df['darshan_biggest_api_bytes'] > 2.0**40, # only want jobs that did more than 1 TiB of I/O
]

pandas.options.display.max_rows = 11
abcutils.apply_filters(df, filters, verbose=True)

Demonstrate how columns can be normalized.  This is the routine used to calculate the fraction of peak performance metric that is calculated for every job.

In [ ]:
target_col = 'darshan_agg_perf_by_slowest_posix'
group_by_cols = ['darshan_app', '_file_system', 'darshan_fpp_or_ssf_job', 'darshan_read_or_write_job']
new_col_base = 'darshan_normalized_perf'

# modifies the dataframe in-place; returns nothing
abcutils.normalize_column(
    dataframe=df,
    target_col=target_col,
    group_by_cols=group_by_cols,
    new_col_base=new_col_base)

In [ ]:
df[group_by_cols + [target_col]].head()

In [ ]:
df[group_by_cols + [target_col] + [new_col_base + "_by_max"]].head()

## Correlation Matrix

In [ ]:
ax, correlations = abcutils.plot.correlation_matrix(df, fontsize=14)
ax.get_figure().set_size_inches(20, 20)

## Correlation Table

In [ ]:
pandas.options.display.max_rows = 60

correlation = abcutils.correlation.calc_correlation_vector(df, correlate_with='darshan_normalized_perf_by_max')

filtered_correlations = abcutils.apply_filters(correlation, [correlation['p-value'] < 1.0e-5], verbose=True)
filtered_correlations

In [ ]:
ax = abcutils.plot.correlation_vector_table(filtered_correlations)
ax.get_figure().set_size_inches(4, 0.4 * len(filtered_correlations))

## Boxplots

In [ ]:
boxplot_settings = {
    'fontsize': 20,
    'darshan_normalized_perf_by_max': {
        'output_file': "perf-boxplots.pdf",
        'ylabel': "Fraction of\nPeak Performance",
        'title_pos': [ 
            {'x': 0.04, 'y': 0.02, 'horizontalalignment': 'left', 'fontsize': 14},
            {'x': 0.04, 'y': 0.02, 'horizontalalignment': 'left', 'fontsize': 14}]
    },
}

Plot a single boxplot using the `plot.grouped_boxplot` function

In [ ]:
ax = abcutils.plot.grouped_boxplot(df[df["_file_system"] == 'scratch1'],
                                   'darshan_normalized_perf_by_max',
                                   fontsize=16)

Boxplots can also be inserted into existing figures with a little more effort.  This can be handy for creating compact publication-ready diagrams.

In [ ]:
NUM_ROWS = 2
NUM_COLS = 2
fig, axes = matplotlib.pyplot.subplots(nrows=NUM_ROWS,
                                       ncols=NUM_COLS,
                                       # sharex causes problems if not all axes contain data
                                       #sharex=True,
                                       sharey=True)
fig.set_size_inches(8,6)

SUBPLOT_ARRANGEMENT = {
    'scratch1': axes[0, 0],
    'scratch2': axes[1, 0],
    'scratch3': axes[1, 1],
}
NULL_SUBPLOTS = [
    axes[0, 1],
]

### Draw subplots that contain data
for index, fs in enumerate(sorted(SUBPLOT_ARRANGEMENT.keys())):
    irow = index / NUM_COLS
    ax = SUBPLOT_ARRANGEMENT[fs]
    abcutils.plot.grouped_boxplot(df[df["_file_system"] == fs],
                                       'darshan_normalized_perf_by_max',
                                       ax=ax,
                                       fontsize=16)
    title = ax.set_title(fs, **(boxplot_settings['darshan_normalized_perf_by_max']['title_pos'][irow]))
    title.set_bbox({'color': 'white', 'alpha': 0.5})

### Hide subplots that do not contain data
for ax in NULL_SUBPLOTS:
    ax.set_visible(False)

### Set global figure labels 
fig.suptitle("")
fig.text(0.0, 0.5,
         boxplot_settings['darshan_normalized_perf_by_max']['ylabel'],
         verticalalignment='center',
         horizontalalignment='center',
         rotation='vertical',
         fontsize=boxplot_settings['fontsize'])
fig.subplots_adjust(hspace=0.05, wspace=0.05)

## Umami Diagrams

In [ ]:
import time
import datetime
import tokio.tools.umami

In [ ]:
umami_diagrams = [
    # The "I/O contention" case study figure
    {
        'filters': [
            df['_file_system'] == 'scratch2',
            df['darshan_app'] == 'hacc_io_write',
            df['darshan_read_or_write_job'] == 'write',
            df['_datetime_start'] > datetime.datetime(2017, 2, 14),
            df['_datetime_start'] < datetime.datetime(2017, 3, 3, 12, 0, 0),
        ],
        'rows': [
            'darshan_agg_perf_by_slowest_posix',
            'coverage_factor_bw',
            'coverage_factor_nodehrs',
            'fs_ave_mds_cpu',
#           'fs_ops_opencloses',
            'topology_job_max_radius',
        ],
    },
    # The "storage capacity" case study figure
    {
        'filters': [
            df['_file_system'] == 'scratch3',
            df['darshan_app'] == 'hacc_io_write',
            df['darshan_read_or_write_job'] == 'write',
            df['_datetime_start'] > datetime.datetime(2017, 2, 21, 0, 0, 0),
            df['_datetime_start'] < datetime.datetime(2017, 3, 15, 0, 0, 0),
        ],
        'rows': [
            'darshan_agg_perf_by_slowest_posix',
            'coverage_factor_bw',
            'coverage_factor_nodehrs',
            'fs_max_oss_cpu',
            'fshealth_ost_most_full_pct',
        ],
    },
]

pandas.options.display.max_rows = 11
filtered_df = abcutils.apply_filters(df, umami_diagrams[0]['filters'], verbose=True)
filtered_df.head().T

In [ ]:
for umami_diagram in umami_diagrams:
    filtered_df = abcutils.apply_filters(df, umami_diagram['filters'], verbose=True)
    fig = abcutils.plot.generate_umami(filtered_df, umami_diagram['rows'])